In [1]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching et3_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s et3_Scene"

Manim Community v0.17.3

Jupyter Capture Output v0.0.11


In [2]:
POLAR_PUPIL_BASE = (0.05, 3/4*PI)


def screen2polar(x, y):
    ppb_radius = POLAR_PUPIL_BASE[0]
    ppb_angle = POLAR_PUPIL_BASE[1]

    correction = 0.8/np.sqrt(5)
    cart_pupil = np.array([ppb_radius*np.cos(ppb_angle) + correction*x, ppb_radius*np.sin(ppb_angle) + correction*y, 0])

    radius = np.sqrt(cart_pupil[0]**2 + cart_pupil[1]**2)
    angle = np.arctan2(cart_pupil[1], cart_pupil[0])
    return (radius, angle)


def screen_path(t):
    if t < np.sqrt(2):
        t /= np.sqrt(2)
        return (-2*t, t)
    elif t < np.sqrt(2) + 2:
        t -= np.sqrt(2)
        return (-2, 1-t)
    elif t < np.sqrt(2) + 2 + 4:
        t -= np.sqrt(2) + 2
        return (-2 + t, -1)
    elif t < np.sqrt(2) + 2 + 4 + 2:
        t -= np.sqrt(2) + 2 + 4
        return (2, -1 + t)
    elif t < 2*np.sqrt(2) + 2 + 4 + 2:
        t -= np.sqrt(2) + 2 + 4 + 2
        t /= np.sqrt(2)
        return (2-2*t, 1-t)
    else:
        return (0, 0)

In [3]:
class Eye(VMobject):
    def __init__(self, position, size, **kwargs):
        super().__init__(**kwargs)
        self.eye_center = position
        self.eye_radius = size
        circle = Circle(radius = self.eye_radius, color = DARK_GREY, stroke_width = 15.0).move_to(self.eye_center)
        self.add(circle)

    # returns the pupil
    def get_pupil(self, pupil_size, pupil_pradius, pupil_angle):
        pupil_radius = pupil_pradius * (self.eye_radius - pupil_size)
        self.pupil_center = self.eye_center + np.array([pupil_radius*np.cos(pupil_angle), pupil_radius*np.sin(pupil_angle), 0])
        pupil_circle = Circle(radius = pupil_size, fill_opacity = 1).move_to(self.pupil_center).set_color(BLACK)
        return pupil_circle
    
    # returns cornea reflection
    def get_reflection(self, reflect_pradius, reflect_angle):
        reflect_radius = reflect_pradius * self.eye_radius
        self.reflect_center = self.eye_center + np.array([reflect_radius*np.cos(reflect_angle), reflect_radius*np.sin(reflect_angle), 0])
        reflect_dot = Dot(point = self.reflect_center, radius = 0.1, fill_opacity = 0.8).set_color([PURE_RED, RED])
        cross_length = 0.75
        reflect_horizontal = Line(start = self.reflect_center - cross_length*RIGHT, end = self.reflect_center + cross_length*RIGHT, stroke_width = 2, color = RED)
        reflect_vertical = Line(start = self.reflect_center - cross_length*UP, end = self.reflect_center + cross_length*UP, stroke_width = 2, color = RED)
        return VGroup(reflect_dot, reflect_horizontal, reflect_vertical)
        #return reflect_dot
    
    def get_connector(self):
        connect_vector = Line(start = self.reflect_center, end = self.pupil_center, color = BLUE).add_tip(tip_length = 0.125, tip_width = 0.125)
        return connect_vector


class Screen(VMobject):
    def __init__(self, position, height, width, **kwargs):
        super().__init__(**kwargs)
        self.npla = NumberPlane(x_range = [-2, 2, 1], y_range = [-1, 1, 0.5], x_length = width*0.8, y_length = height*0.8).move_to(position)
        #self.add(npla)
        rectangle = Rectangle(height = height, width = width, stroke_width = 4, color = BLACK).move_to(position)
        self.add(rectangle)

        dot_center = Dot(self.npla.c2p(0, 0, 0), radius = 0.1, color = GREY)
        dot_top_left = Dot(self.npla.c2p(-2, 1, 0), radius = 0.1, color = GREY)
        dot_bottom_left = Dot(self.npla.c2p(-2, -1, 0), radius = 0.1, color = GREY)
        dot_bottom_right = Dot(self.npla.c2p(2, -1, 0), radius = 0.1, color = GREY)
        dot_top_right = Dot(self.npla.c2p(2, 1, 0), radius = 0.1, color = GREY)
        self.add(dot_center, dot_top_left, dot_bottom_left, dot_bottom_right, dot_top_right)

    
    def get_viewpoint(self, x, y):
        dot_viewpoint = Dot(self.npla.c2p(x, y, 0), radius = 0.075, color = RED, fill_opacity = 0.75)
        return dot_viewpoint

In [13]:
%%capture_video --path "animations/eye_tracking_3/eye_tracking_3_START.mp4"
%%manim -qh --fps 60 $video_scene


class et3_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        # human eye to graphic transition
        human_eye = ImageMobject("external_media/Eye1.jpg")
        self.add(human_eye)

        eye = Eye(position = np.array([0, 0.25, 0]), size = 2.15)
        pupil = eye.get_pupil(pupil_size = 0.38, pupil_pradius = POLAR_PUPIL_BASE[0], pupil_angle = POLAR_PUPIL_BASE[1])
        cornea_reflection = eye.get_reflection(reflect_pradius = 0.5, reflect_angle = 7/4*PI)
        vector = eye.get_connector()

        # eye tracking calibration
        screen = Screen(position = np.array([2, 0.25, 0]), height = 5*0.8, width = 8*0.8)
        viewpoint = screen.get_viewpoint(0, 0)

        # MAIN
        self.add(human_eye)
        self.wait(1.5)
        self.play(FadeIn(eye), FadeIn(pupil), FadeOut(human_eye), run_time = 3)
        self.wait(0.5)
        self.play(FadeIn(cornea_reflection), run_time = 3)
        self.play(Create(vector), run_time = 1.5)
        self.wait(1.5)

        eye1 = VGroup(eye, pupil, cornea_reflection, vector)
        self.play(eye1.animate.scale(0.5).shift(4*LEFT), vector.tip.animate.shift(4*LEFT), run_time = 1.5)
        self.wait(1.5)

        self.play(Create(screen), run_time = 4.5)
        self.wait(0.5)
        self.play(FadeIn(viewpoint), run_time = 1.5)
        self.wait(1.5)

Output saved by overwring previous file at animations/eye_tracking_3/eye_tracking_3_START.mp4.


In [15]:
%%capture_video --path "animations/eye_tracking_3/eye_tracking_3_CALIBRATION.mp4"
%%manim -qh --fps 60 $video_scene


class et3_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE


        eye = Eye(position = np.array([0, 0.25, 0] + 4*LEFT), size = 2.15*0.5)
        self.add(eye)

        pupil = eye.get_pupil(pupil_size = 0.38*0.5, pupil_pradius = POLAR_PUPIL_BASE[0], pupil_angle = POLAR_PUPIL_BASE[1])
        self.add(pupil)

        cornea_reflection = eye.get_reflection(reflect_pradius = 0.5, reflect_angle = 7/4*PI).scale(0.5)
        self.add(cornea_reflection)

        vector = eye.get_connector()
        self.add(vector)


        # eye tracking calibration
        screen = Screen(position = np.array([2, 0.25, 0]), height = 5*0.8, width = 8*0.8)
        self.add(screen)

        viewpoint = screen.get_viewpoint(0, 0)
        self.add(viewpoint)
        
        
        def viewpoint_updater(point):
            t = t_tracker.get_value()
            x, y = screen_path(t)
            rradius, angle = screen2polar(x, y)

            point.become(screen.get_viewpoint(x, y))

        def pupil_updater(pupil):
            t = t_tracker.get_value()
            x, y = screen_path(t)
            rradius, angle = screen2polar(x, y)

            pupil.become(eye.get_pupil(pupil_size = 0.38*0.5, pupil_pradius = rradius, pupil_angle = angle))
            vector.become(eye.get_connector())  


        # CALIBRATION ANIMATION
        t_tracker = ValueTracker(0)
        viewpoint.add_updater(viewpoint_updater)
        pupil.add_updater(pupil_updater)
        self.play(t_tracker.animate.set_value(12), rate_func = linear, run_time = 10)
        self.wait(3)
    

Output saved by overwring previous file at animations/eye_tracking_3/eye_tracking_3_CALIBRATION.mp4.


In [ ]:
#os.system('ffmpeg -f concat -i line_integration_merge_list.txt -c copy line_integration_FULL_1.mp4')